# Variational autoencoder
This notebook will investiga

In [1]:
%matplotlib
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from variational_autoencoder import VariationalAutoencoder

# For dimension reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# for visualization and evaluation
from latent_plane_mosaic import LatentPlaneMosaic
from latent_interpolation_mosaic import LatentInterpolationMosaic
from sample_scatter_gui import SampleScatterGUI

Using matplotlib backend: Qt5Agg


In [2]:
# Global settings
model_path = "Models/VAE_cnn_deep_latent_dim_8"
num_samples = 20000
np.set_printoptions(precision=0)

## Preprocess data


In [50]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
mnist_digits = np.concatenate([x_train, x_test], axis=0)
mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255
mnist_labels = np.concatenate([y_train, y_test], axis=0)
input_shape = mnist_digits.shape[1:]
num_digits = len(mnist_labels)
sample_index = rnd.randint(0, num_digits, size = (num_samples,))
mnist_digits = mnist_digits[sample_index]
mnist_labels = mnist_labels[sample_index]

## Load Variational autoencoder

In [51]:
# Variational autoencoder
model = keras.models.load_model(model_path, 
                         custom_objects={"VariationalAutoencoder": VariationalAutoencoder})


### Print setup details

In [52]:
print("Input shape: ", input_shape)
print("Latent dim: ",  model.latent_dim)
print("Num parameters: ", model.count_params())



Input shape:  (28, 28, 1)
Latent dim:  8
Num parameters:  367605


## Reconstruct and dimension reductions

In [53]:
# Reconstructions
mnist_encoded = model.encode(mnist_digits).numpy()

# t-SNE
mnist_tsne2d = TSNE(n_components = 2).fit_transform(mnist_encoded)

#pca
pca = PCA().fit(mnist_encoded)
mnist_pca = pca.transform(mnist_encoded)
latent_cov = np.cov(mnist_pca.T)
latent_std = np.sqrt(pca.explained_variance_)
latent_axes = pca.components_
latent_mean = pca.mean_


## Investigation

### Show reconstructions

In [54]:
num_col = 4
num_row = 4
num_img = num_row*num_col
fig, axs = plt.subplots(num_row, num_col)

rec_index = rnd.randint(num_samples,size =(num_row*num_col,) )
digits = mnist_digits[rec_index]
reconstructions = model(digits)
i = 0
for col in range(num_col):
    for row in range(num_row): 
        ax = axs[row, col]
        im = np.concatenate((digits[i],
                             reconstructions[i]),
                             axis = 1)
        ax.imshow(im)
        ax.set_title("Reconstruction")
        i = i +1

### Latent

In [55]:
indeces = rnd.randint(num_samples,size = (3,))

ul = mnist_digits[indeces[0]]
ur = mnist_digits[indeces[1]]
dl = mnist_digits[indeces[2]]
z =  np.zeros(dl.shape)
corner_image = np.concatenate( (np.concatenate((ul,ur),axis = 1),
                                np.concatenate((dl,z ),axis = 1)), axis = 0)

mosaic = LatentInterpolationMosaic(
                          model.encode,
                          model.decoder,
                          mnist_digits,
                          indeces,
                          num_row = 15,
                          num_col = 15).mosaic





fig, axs = plt.subplots(1,2)
axs[0].imshow(corner_image)
axs[1].imshow(mosaic)

### Scatter

In [56]:
SampleScatterGUI(mnist_tsne2d, mnist_labels, mnist_digits)
#if latent dim == 2
#SampleScatterGUI(mnist_encoded, mnist_labels, mnist_digits)

### Spread

In [57]:
fig, ax = plt.subplots(1,1)
ax.plot(latent_std, 'o--')
ax.set_yscale("log")
ax.set_xlabel("Principal axis index in decreasing order")
ax.set_ylabel("Principal variance for the corresponding axis")
ax.set_title("Principal variance")

Text(0.5, 1.0, 'Principal variance')

In [45]:
c = latent_cov-np.diag(np.diag(latent_cov))
print("Max non-diagonal covariance ", np.max(np.abs(c)))

Max non-diagonal covariance  7.453978385330668e-07


Here we have that the first 6 principal axises are almost one and the remaining 4 are almost zero (e-3). This is combination with that the cross covariance seem to be super much zero (e-15), indicates that we have a six dimesnioal sphere in this ten dimesional latent space. Interesting to see is also if you create a latent vector in the pca-vector space. Then comparing the latent value in each dimension times the spread in each dimension, gives you the importance of that dimension in that latent point. A low importance will not have an effect on the output (kind of like a low derivative...), while a hight value will give a big difference.

In [47]:
axes_index = [2,3]
latent_vectors = latent_axes[axes_index]
latent_origin = latent_mean
scaling_factors = np.array([5,5])

mosaic = LatentPlaneMosaic(model.decoder,
                  latent_vectors = (scaling_factors*latent_vectors.T).T,
                  latent_origin = latent_origin,
                  num_row = 20,
                  num_col = 20).mosaic

fig,ax = plt.subplots(1,1)
ax.imshow(mosaic)


In [13]:
np.linalg.norm(latent_vectors, axis = 1)

array([1., 1.], dtype=float32)

In [21]:
latent_origin

array([ 0., -0., -0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,
        0.,  0.,  0.], dtype=float32)

In [59]:
latent_axes*1000


array([[ 3.e+02,  6.e+01,  3.e+02,  3.e+00, -1.e+02,  3.e+02, -8.e+02,
        -6.e+01],
       [ 2.e+02, -4.e+01,  5.e+02,  2.e+00,  1.e+02,  7.e+02,  5.e+02,
         4.e+01],
       [-7.e+02, -4.e+01,  7.e+02, -6.e-01,  1.e+02, -3.e+02, -8.e+01,
         4.e+01],
       [ 4.e+02,  2.e+02,  4.e+02,  2.e+00, -6.e+02, -4.e+02,  3.e+02,
        -2.e+02],
       [ 5.e+02, -3.e+02,  2.e+02, -1.e-01,  6.e+02, -4.e+02,  7.e+00,
         2.e+02],
       [-5.e+00, -5.e+02, -2.e+00,  2.e+02, -5.e+02,  8.e+00, -6.e+00,
         7.e+02],
       [ 8.e-01, -8.e+02,  6.e+00, -2.e+02, -9.e+01, -7.e-01,  3.e+00,
        -6.e+02],
       [-1.e+00, -8.e+01, -1.e-01,  1.e+03,  7.e+01, -4.e+00,  3.e+00,
        -3.e+02]], dtype=float32)

In [64]:
model.built

True

In [65]:
model.summary()

Model: "variational_autoencoder_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder (Sequential)         (None, 2, 8)              210580    
_________________________________________________________________
Decoder (Sequential)         (None, 28, 28, 1)         157017    
_________________________________________________________________
normal_sampling_layer_3 (Nor multiple                  0         
Total params: 367,605
Trainable params: 367,597
Non-trainable params: 8
_________________________________________________________________
